# Use fastgather to create a manifest for mgmanysearch
mgmanysearch will be faster if it has to search through less results
Can I make only 1 manifest for vOTUs and MAGs, and then later run it against the db of interest? 
So that would be 99%, 95% and all.

Try first for one metagenome, if that works then rewrite the snakefile so that it happesn automatically

In [ ]:
# for the votus, try with scale of 100, resketch the reads using the sketch snakefile
snakemake -s Snakefile_sketch \
--resources mem_mb=30000 \
--rerun-triggers mtime -c 60 --rerun-incomplete -k -p --unlock

# vOTUs are already sketchet at 100, in sig_files/signatures_concat/vOTUs.k21.s100.zip

# run fmg
srun --account=ctbrowngrp -p bmm -J sk_read -t 30:00:00 -c 60 --mem=30gb --pty bash

mamba activate branchwater

sourmash scripts fastmultigather \
../all_reads.txt \
../../sig_files/signatures_concat/vOTUs.k21.s100.zip \
-c 48 -k 21 -t 5000 -s 100

In [ ]:
# run fastgather in the sourmash folder, via snakemake
sourmash scripts fastgather \
{input.mtgreads} {input.query} -k 21 --scaled 1000 \
-o {output.csv} 

# run snakefile:
snakemake -s Snakefile_mgsearch \
--resources mem_mb=30000 --rerun-triggers mtime -c 36 --rerun-incomplete -k -p 

In [ ]:
# after fastgather, make a manifest from the results

# get the fastgather column 'match_name'
mamba activate csvtk
csvtk cut -f match_name SRR8960970.fg.csv > ../../../manifests/vOTU/SRR8960970.list.csv

cd ../../../manifests/vOTU

In [ ]:
# remove header
csvtk del-header SRR8960970.list.csv > SRR8960970.list.dh.csv 

# now we need to find a way to make manifests for all/99/95 so cross compare
# sort the list
sort -u SRR8960970.list.dh.csv > SRR8960970.all.csv 

# then compare
comm -1 -2 SRR8960970.all.csv \
../../../genome_stats/240215_high_qualityvOTUs95.nc.sort.txt \
> SRR8960970.95.csv

comm -1 -2 SRR8960970.all.csv \
../../../genome_stats/240215_high_qualityvOTUs99.nc.sort.txt \
> SRR8960970.99.csv

# add the file path to all 3 in place
sed -i -e 's#^#/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/sig_files/sketch_vOTUs/high_qual/#' SRR8960970.all.csv 
sed -i -e 's#^#/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/sig_files/sketch_vOTUs/high_qual/#' SRR8960970.99.csv 
sed -i -e 's#^#/group/ctbrowngrp2/scratch/annie/2023-swine-sra/sourmash/sig_files/sketch_vOTUs/high_qual/#' SRR8960970.95.csv 

# add sig.gz
awk 'NF{print $0 ".sig.gz"}' SRR8960970.all.csv > SRR8960970.all.sig.csv 
awk 'NF{print $0 ".sig.gz"}' SRR8960970.99.csv > SRR8960970.99.sig.csv 
awk 'NF{print $0 ".sig.gz"}' SRR8960970.95.csv > SRR8960970.95.sig.csv 

# sourmash manifest 
sourmash sig collect $(cat SRR8960970.all.sig.csv ) -o SRR8960970.all.mf.csv -F csv
sourmash sig collect $(cat SRR8960970.99.sig.csv ) -o SRR8960970.99.mf.csv -F csv
sourmash sig collect $(cat SRR8960970.95.sig.csv ) -o SRR8960970.95.mf.csv -F csv

In [ ]:
# then run mgmanysearch
sourmash scripts mgmanysearch 
--queries SRR8960970.all.mf.csv --against SRR8960970.reads.sig \
-k 21 --scaled 1000 -o SRR8960970.all.mg.csv


In [ ]:
# run prodigal on vOTUs so we can sketch protein and see how many hashes we lose with deduplication
# in ./virsorter2/
mamba activate prodigal
for f in 240214*.fa
do 
prodigal -i $f -a ../protein_files/${f%.fa*}.faa -p meta -q
done

# Then sourmash sketch the files at k=10, scaled=100
for f in *.faa
do
sourmash sketch protein -p k=7,scaled=100,noabund $f \
-o ../../sourmash/sig_files/prot_sketches/$f.k7.zip
done

# Then do sig fileinfo to check how large they are. 
for f in *k7.zip
do
sourmash sig fileinfo $f
done

In [ ]:
# resketch all MAGs so that all files have a name
# in atlas/MAGs/genomes/all_fasta/fastafiles/

srun --account=ctbrowngrp -p med2 -J smash -t 24:00:00 -c 1 --mem=30gb --pty bash
module load parallel
mamba activate sourmash 
for f in *.fasta
do
sourmash sketch dna \
-p k=21,scaled=1000,k=31,scaled=1000,k=51,scaled=1000 \
$f --name ${f%.fasta*} -o \
../../../../../sourmash/sig_files/MAGs2.0/${f%.fasta*}.zip
done | parallel -j 32

# move/copy files from a list into a new folder
 for f in $(cat MAGSdone.txt); do mv $f.fasta ./smash_done/; done

# Concatenate all signatures
sourmash sig cat *.zip -o ../signatures_concat/MAGs2.zip && \
sourmash sig collect *.zip -o ../signatures_concat/MAGs2.sqlmf

Index to rocksdb for fastmultigather?
sourmash scripts index MAGs2.zip -o MAGs2.rocksdb -k 21 -c 32

In [ ]:
# try fastmultigather -> picklist -> mgsearch
# in /sourmash/mgmanysearch/fastmultigather

sourmash scripts fastmultigather \
../all_reads.txt \
../../sig_files/signatures_concat/MAGs2.zip \
-c 36 -k 21

# set len treshold at half length of smallest predicted vOTU. 
sourmash scripts fastmultigather \
../all_reads.txt \
../../sig_files/signatures_concat/allvOTUs.zip \
-c 36 -k 21 -t 5000 -s 100


# The OG manifest has some columns where name is empty.


# cut md5sum col
csvtk cut -f match_md5 ./allMAGs/SRR9323811.fg.csv > SRR9323811.pl.txt

# for OG MAGs (with no names)
sourmash sig check ../../sig_files/signatures_concat/allMAGs.zip \
--picklist SRR9323811.pl.txt:match_md5:md5 -m SRR9323811.mf.og.csv

# then make mfs for 95,99,all
csvtk join -f "1;name" ../../genome_stats/genome_lists/MAGs.95.txt mf.csv > test.csv

# do again with this db, check if name is still empty
sig_files/signatures_concat/MAGs2.zip

In [ ]:
# fedF operon (in folder fed_operon)
# copy all assemblies so we can sketch them all

# sketch the fedF file (8000 bp)
sourmash sketch dna scaffold00000063_fed_genes_ETEC.fa -o fed_genes_ETEC.zip -k 21 --scaled 1000

# then try against genomes, assemblies and reads:
sourmash gather AtH2023_SRR12795783_MAG2.sig.gz \
../../../../fed_operon/fed_genes_ETEC.zip -k 21 --scaled 1000 \
-o ../../../../fed_operon/AtH2023_SRR12795783_MAG2.csv --threshold-bp 100

# sketch assemblies
for f in ../../../atlas/atlas_*/*/*_contigs.fa
do
echo sourmash sketch dna -p k=21,scaled=1000 $f --name ${f%.fa*} -o ${f%.fa*}.zip
done | parallel -j 32